In [40]:
import torch
from ganpaint import nethook, setting, renormalize, zdataset, widget
from ipywidgets import Output
# Load a generator, and paint random image number 20 in a widget.
G = nethook.InstrumentedModel(setting.load_proggan('church')).cuda()
z = zdataset.z_sample_for_model(G)[60][None].cuda()
V = widget.PaintWidget(image=renormalize.as_url(G(z)[0]))

# Whenever the widget changes, call "changed" to edit the layer and repaint
TREE_UNITS = [119, 125, 157, 195, 365, 374, 418, 426, 408, 3, 297, 322, 343]
WINDOW_UNITS = [189, 279, 435, 203, 236, 334]
GRASS_UNITS = [14, 159, 125, 380]
out = Output()
display(out)

# TREE_UNITS = [119]
current_mask = torch.zeros(1, 512)
saved_a = None
@out.capture()
def changed(c):
    global saved_a
    if not V.mask: return
    a = renormalize.from_url(V.mask, target='pt', size=(8,8))[0]
    saved_a = a
    def editrule(x, imodel, **buffers):
        x[:,GRASS_UNITS] += (a * 50.0).to(x.device)
        return x
    G.edit_layer('layer4', rule=editrule)
    V.mask, V.image = '', renormalize.as_url(G(z)[0])
    mage = renormalize.as_url(G(z)[0])
V.observe(changed, names='mask')

display(V)

<IPython.core.display.Javascript object>

Output()

PaintWidget(image='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAIAAADTED8xAAEAAElEQVR4nIz9Z7hlyXUYh…

In [29]:
saved_a

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [16]:
import torch
from ganpaint import nethook, setting, renormalize, zdataset, widget

# Load a generator, and paint random image number 20 in a widget.
G = nethook.InstrumentedModel(setting.load_proggan('bedroom')).cuda()
z = zdataset.z_sample_for_model(G)[10][None].cuda()
V = widget.PaintWidget(image=renormalize.as_url(G(z)[0]))

# Whenever the widget changes, call "changed" to edit the layer and repaint
# TREE_UNITS = [119, 125, 157, 195, 365, 374, 418, 426, 408, 3, 297, 322, 343]
# LAMP_UNITS = [136, 502, 4, 462, 206, 287, 222, 163, 58, 182, 432, 231, 494, 449]
# LAMP_UNITS = list(range(50,100))
LAMP_UNITS = [462]
current_mask = torch.zeros(1, 512)
def changed(c):
    if not V.mask: return
    a = renormalize.from_url(V.mask, target='pt', size=(8,8))[0]
    def editrule(x, imodel, **buffers):
        x[:,LAMP_UNITS] += (a * 50.0).to(x.device)
        # x[:,LAMP_UNITS] = x[:,LAMP_UNITS] * (1 - a).to(x.device)
        return x
    G.edit_layer('layer4', rule=editrule)
    V.mask, V.image = '', renormalize.as_url(G(z)[0])
V.observe(changed, names='mask')

display(V)

<IPython.core.display.Javascript object>

PaintWidget(image='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAIAAADTED8xAAEAAElEQVR4nLT9WZMsy5Yeh…

In [12]:
V


PaintWidget(image='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAIAAADTED8xAAEAAElEQVR4nGz9aZBlS5Ieh…